In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Tuning with Vertex AI Foundation Model

LLMOps, or Large Language Model Operations, is an important methodology as organizations increasingly adopt large language models (LLMs) for a wide range of applications. LLMOps is the set of tools, processes, and best practices for managing the lifecycle of LLMs, from development and deployment to monitoring and maintenance. Vertex AI offers services to manage LLMOps pipelines as also mechanisms to evaluate the new models quality after every pipeline execution that you run. Model fine-tuning is a powerful technique used to improve the performance of pre-trained language models (LLMs) for specific tasks or domains. It involves adjusting the model's parameters based on a new dataset or task-specific data to enhance its ability to make accurate predictions or generate relevant text. By fine-tuning an LLM, we can leverage its existing knowledge and adapt it to a specific context, resulting in improved results and better-tailored outputs. For more details on tuning have a look at the official [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-models).

# Objective

This lab teaches you how to tune a foundational model on new unseen data and you will use the following Google Cloud products:
*   Vertex AI Pipelines
*   Vertex AI Evaluation Services
*   Vertex AI Model Registry
*   Vertex AI Endpoints

# Use Case

Using Generative AI we will generate a suitable TITLE for a news BODY from BBC FULLTEXT DATA (Sourced from BigQuery Public Dataset *bigquery-public-data.bbc_news.fulltext*). We will fine tune Gemini 1.5 Pro to a new fine-tuned model called "bbc-news-summary-tuned" and compare the result with the response from the base model.

# Install and Import Dependencies

In [ ]:
!pip install google-cloud-aiplatform
!pip install --user datasets
!pip install --user google-cloud-pipeline-components

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import IPython
from google.cloud import aiplatform
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai
PROJECT_ID = "YOUR PROJECT ID" #@param
vertexai.init(project=PROJECT_ID)

In [ ]:
region = "us-central1"
REGION = "us-central1"
project_id = PROJECT_ID

In [ ]:
! gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#Import the necessary libraries

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)
import kfp
import sys
import uuid
import json
import vertexai
import pandas as pd
from google.auth import default
from datasets import load_dataset
from google.cloud import aiplatform
from vertexai.preview.language_models import TextGenerationModel, EvaluationTextSummarizationSpec


# Prepare & Load Training Data

In [ ]:
BUCKET_NAME = 'llm_tuning_ssn6_a'
BUCKET_URI = f"gs://llm_tuning_ssn6_a/TRAIN.jsonl"
REGION = "us-central1"

In [ ]:
json_url = 'https://storage.googleapis.com/img_public_test/next_demo/TRAIN.jsonl'
df = pd.read_json(json_url, lines=True)
print (df)

In [ ]:
print(df.shape)

(744, 2)


In [ ]:
from vertexai.preview import tuning
from vertexai.preview.tuning import sft

In [ ]:
eval_json_url = 'https://storage.googleapis.com/img_public_test/next_demo/EVALUATE.jsonl'
eval_df = pd.read_json(eval_json_url, lines=True)
print (eval_df)

In [ ]:
sft_tuning_job = sft.train(
      source_model="gemini-1.0-pro-002",
      train_dataset=str('gs://img_public_test/next_demo/TRAIN.jsonl'),
      validation_dataset=str('gs://img_public_test/next_demo/EVALUATE-100.jsonl'),
      epochs=3,
      learning_rate_multiplier=1,
  )

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/273845608377/locations/us-central1/tuningJobs/8460302892639715328
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/273845608377/locations/us-central1/tuningJobs/8460302892639715328')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/8460302892639715328?project=273845608377


# Predict with the new Fine Tuned Model

In [ ]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview import tuning
from vertexai.preview.tuning import sft

sft_tuning_job = sft.SupervisedTuningJob("projects/273845608377/locations/us-central1/tuningJobs/8460302892639715328")
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)
content = "Summarize this text to generate a title: \n Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable it it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans."
print(tuned_model.generate_content(content).text)

Are tight aeroplane seats putting passengers at risk?


# Predict with Base Model for comparison

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [ ]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


def generate():
  vertexai.init(project=project_id, location="us-central1")
  model = GenerativeModel(
    "gemini-1.0-pro-002",
  )
  text1 = "Summarize this text to generate a title: \n Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable it it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans."
  responses = model.generate_content(
      [text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")


generate()

## Passengers vs. Profit: Are Shrinking Plane Seats Putting Our Health and Safety at Risk? 
